In [4]:
!pip install ipdb

  Running setup.py bdist_wheel for ipdb ... done
  Stored in directory: /Users/hiroakioshima/Library/Caches/pip/wheels/59/24/91/695211bd228d40fb22dff0ce3f05ba41ab724ab771736233f3
Successfully built ipdb


In [252]:
import torch
import torch.nn as nn
from torch.utils import data
from mds189 import Mds189
import numpy as np
from skimage import io, transform
import ipdb
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision.models as models
from PIL import Image
import time
start = time.time()

In [253]:
# Helper functions for loading images.
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)

def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)

In [254]:
# flag for whether you're training or not
is_train = True
is_key_frame = True # TODO: set this to false to train on the video frames, instead of the key frames
model_to_load = 'model.ckpt' # This is the model to load during testing, if you want to eval a previously-trained model.


In [255]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#cudnn.benchmark = True

### Batch Size

In [256]:
# Parameters for data loader
params = {'batch_size': 32,  # TODO: fill in the batch size. often, these are things like 32,64,128,or 256
          'shuffle': True,
          'num_workers': 2 
          }


### hyper parameters

In [257]:
# TODO: Hyper-parameters
num_epochs = 5
learning_rate = 0.1
# NOTE: depending on your optimizer, you may want to tune other hyperparameters as well

### datasets

In [258]:
# Datasets
# TODO: put the path to your train, test, validation txt files
if is_key_frame:
    label_file_train =  'dataloader_files/keyframe_data_train.txt'
    label_file_val  =  'dataloader_files/keyframe_data_val.txt'
    # NOTE: the kaggle competition test data is only for the video frames, not the key frames
    # this is why we don't have an equivalent label_file_test with keyframes
else:
    label_file_train = '../data/hw6_mds189/videoframe_data_train.txt'
    label_file_val = '../data/hw6_mds189/videoframe_data_val.txt'
    label_file_test = '../data/hw6_mds189/videoframe_data_test.txt'

### normalization

In [259]:
# TODO: you should normalize based on the average image in the training set. This shows 
# an example of doing normalization
mean = [134.010302198/255.0, 118.599587912/255.0, 102.038804945/255.0]
std = [23.5033438916/255.0, 23.8827343458/255.0, 24.5498666589/255.0]

### padding & resize

In [271]:
# TODO: if you want to pad or resize your images, you can put the parameters for that below.
# transforms.Pad(padding=0, fill=0), # TODO: if you want to pad your images
# Generators
# NOTE: if you don't want to pad or resize your images, you should delete the Pad and Resize
# transforms from all three _dataset definitions.
train_dataset = Mds189(label_file_train,loader=default_loader,transform=transforms.Compose([
                                               transforms.Resize([227,227]), # TODO: if you want to resize your images
                                               transforms.Grayscale(3), 
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean, std)
                                           ]))
train_loader = data.DataLoader(train_dataset, **params)

val_dataset = Mds189(label_file_val,loader=default_loader,transform=transforms.Compose([
                                               transforms.Resize([227, 227]),
                                               transforms.Grayscale(3), 
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean, std)
                                           ]))
val_loader = data.DataLoader(val_dataset, **params)

if not is_key_frame:
    test_dataset = Mds189(label_file_test,loader=default_loader,transform=transforms.Compose([
                                                   transforms.Resize([227, 227]),
                                                   transforms.Grayscale(3), 
                                                   transforms.ToTensor(),
                                                   transforms.Normalize(mean, std)
                                               ]))
    test_loader = data.DataLoader(test_dataset, **params)


In [272]:
train_dataset[0][0].size()


torch.Size([3, 227, 227])

In [273]:
# TODO: one way of defining your model architecture is to fill in a class like NeuralNet()
# NOTE: you should not overwrite the models you try whose performance you're keeping track of.
#       one thing you could do is have many different model forward passes in class NeuralNet()
#       and then depending on which model you want to train/evaluate, you call that model's
#       forward pass. this strategy will save you a lot of time in the long run. the last thing
#       you want to do is have to recode the layer structure for a model (whose performance
#       you're reporting) because you forgot to e.g., compute the confusion matrix on its results
#       or visualize the error modes of your (best) model
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        # you can define some common layers, for example: 
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 8),
        )

    def forward(self, x):
        # now you can use the layers you defined, to write the forward pass, i.e.,
        # network architecture for your model
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x


### VGG16

In [274]:
model = NeuralNet().to(device)

In [275]:
# if we're only testing, we don't want to train for any epochs, and we want to load a model
if not is_train:
    num_epochs = 0
    model.load_state_dict(torch.load('model.ckpt'))

### Criterion

In [276]:
# Loss and optimizer
criterion =  torch.nn.CrossEntropyLoss().cuda() #TODO: define your loss here. hint: should just require calling a built-in pytorch layer.
# NOTE: you can use a different optimizer besides Adam, like RMSProp or SGD, if you'd like
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [277]:
for i, (local_batch,local_labels) in enumerate(train_loader):
    print(local_labels)
    if i == 3:
        break

tensor([5, 6, 3, 3, 5, 1, 6, 5, 7, 4, 6, 6, 4, 7, 3, 6, 3, 4, 5, 6, 4, 6, 5, 4,
        5, 4, 4, 2, 7, 6, 6, 5])
tensor([3, 2, 2, 4, 5, 1, 5, 6, 1, 6, 0, 6, 4, 4, 6, 6, 0, 4, 3, 3, 4, 6, 2, 3,
        2, 5, 5, 4, 5, 6, 1, 6])
tensor([4, 3, 6, 1, 3, 2, 7, 1, 6, 7, 4, 3, 4, 5, 0, 5, 1, 5, 2, 6, 5, 3, 7, 2,
        7, 6, 6, 6, 6, 7, 5, 3])
tensor([6, 3, 5, 2, 6, 3, 3, 3, 4, 5, 6, 6, 2, 5, 1, 2, 2, 5, 6, 3, 4, 6, 3, 4,
        4, 6, 1, 5, 3, 2, 5, 3])


In [278]:
# Train the model
# Loop over epochs
def train_model(model, optimizer, criterion, learning_rate, num_epochs, decay_rate=1, decay_start=3):
    print('Beginning training..')
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        # Training
        if epoch >= decay_start:
            learning_rate = learning_rate * decay_rate
            optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        print('epoch {}'.format(epoch))
        for i, (local_batch,local_labels) in enumerate(train_loader):
            # Transfer to GPU
            local_ims, local_labels = local_batch.to(device), local_labels.to(device)

            # Forward pass
            outputs = model.forward(local_ims)
            loss = criterion(outputs, local_labels)
            # TODO: maintain a list of your losses as a function of number of steps
            #       because we ask you to plot this information
            # NOTE: if you use Google Colab's tensorboard-like feature to visualize
            #       the loss, you do not need to plot it here. just take a screenshot
            #       of the loss curve and include it in your write-up.
            # loss_list =
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 4 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    end = time.time()
    print('Time: {}'.format(end - start))

In [279]:
model = NeuralNet().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
train_model(model, optimizer, criterion, 0.1, 8, decay_rate=0.5, decay_start=3)


Beginning training..
epoch 0
Epoch [1/8], Step [4/91], Loss: 2.0747
Epoch [1/8], Step [8/91], Loss: 2.0569
Epoch [1/8], Step [12/91], Loss: 2.0406
Epoch [1/8], Step [16/91], Loss: 2.0605
Epoch [1/8], Step [20/91], Loss: 2.1084
Epoch [1/8], Step [24/91], Loss: 2.0260
Epoch [1/8], Step [28/91], Loss: 2.0544
Epoch [1/8], Step [32/91], Loss: 1.9621
Epoch [1/8], Step [36/91], Loss: 2.0264
Epoch [1/8], Step [40/91], Loss: 2.0100
Epoch [1/8], Step [44/91], Loss: 1.9961
Epoch [1/8], Step [48/91], Loss: 2.0442
Epoch [1/8], Step [52/91], Loss: 1.9665
Epoch [1/8], Step [56/91], Loss: 1.9821
Epoch [1/8], Step [60/91], Loss: 1.9738
Epoch [1/8], Step [64/91], Loss: 2.0176
Epoch [1/8], Step [68/91], Loss: 2.0223
Epoch [1/8], Step [72/91], Loss: 1.9585
Epoch [1/8], Step [76/91], Loss: 2.0022
Epoch [1/8], Step [80/91], Loss: 2.0974
Epoch [1/8], Step [84/91], Loss: 1.9828
Epoch [1/8], Step [88/91], Loss: 1.9703
epoch 1
Epoch [2/8], Step [4/91], Loss: 1.8846
Epoch [2/8], Step [8/91], Loss: 2.0777
Epoch [

In [282]:
output_list = []
labels_list = []
for i, (val_batch,val_labels) in enumerate(val_loader):
    val_ims, val_labels = val_batch.to(device), val_labels.to(device)
    outputs = model.forward(val_ims)
    labels = outputs.argmax(1)
    output_list.append(labels)
    labels_list.append(val_labels)
    print(i)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [283]:
from sklearn.metrics import accuracy_score
output_labels = torch.cat(output_list)
val_labels = torch.cat(labels_list)
accuracy_score(output_labels, val_labels)

0.7302564102564103

the best so far :
train_model(model, optimizer, criterion, 0.1, 8, decay_rate=0.5) with decay >=3
with gray scale

### validation & accuracy

In [66]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
print('Beginning Testing..')
with torch.no_grad():
    correct = 0
    total = 0
    predicted_list = []
    groundtruth_list = []
    for (local_batch,local_labels) in test_loader:
        # Transfer to GPU
        local_ims, local_labels = local_batch.to(device), local_labels.to(device)

        outputs = model.forward(local_ims)
        _, predicted = torch.max(outputs.data, 1)
        total += local_labels.size(0)
        predicted_list.extend(predicted)
        groundtruth_list.extend(local_labels)
        correct += (predicted == local_labels).sum().item()

    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))

# Look at some things about the model results..
# convert the predicted_list and groundtruth_list Tensors to lists
pl = [p.cpu().numpy().tolist() for p in predicted_list]
gt = [p.cpu().numpy().tolist() for p in groundtruth_list]


Beginning Testing..


NameError: name 'test_loader' is not defined

In [ ]:
# TODO: use pl and gt to produce your confusion matrices

# view the per-movement accuracy
label_map = ['reach','squat','inline','lunge','hamstrings','stretch','deadbug','pushup']
for id in range(len(label_map)):
    print('{}: {}'.format(label_map[id],sum([p and g for (p,g) in zip(np.array(pl)==np.array(gt),np.array(gt)==id)])/(sum(np.array(gt)==id)+0.)))


In [ ]:
# TODO: you'll need to run the forward pass on the kaggle competition images, and save those results to a csv file.
if not is_key_frame:
    # your code goes here!
    pass


In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')